# 1) Code for Printing mysql DataBase Schema

In [1]:
import mysql.connector

Schema = ""
col = []

database='nepa'

# Establish a connection to the database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='nepa'
)


cursor = connection.cursor()

# Fetch the list of tables in the database
dtables = []
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

print(f"In Our DataBase {database} we have Following Tables:")
for i in tables:
    print("Table:", i[0])
    dtables.append(i[0])

# Initialize the actual_cols to store the list of column names for each table
actual_cols = []
tabdata = []

for table_name in dtables:
    cursor.execute(f"DESCRIBE {table_name}")
    des = cursor.fetchall()

    # Collect column names for the current table
    cols = [column[0] for column in des]
    actual_cols.append(cols)
    
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = cursor.fetchall()
    tabdata.append(rows)

# Loop through each table and describe its schema
for table_name in dtables:
    Schema += f"\nSchema of table `{table_name}`:\n"
    cursor.execute(f"DESCRIBE {table_name}")
    columns = cursor.fetchall()
    
    # Print the header for the schema
    Schema += f'Description of Table {table_name}["Field","Type","Null","Key","Default","Extra"] are given below ==> \n\n'
    
    # Print each column's schema
    for column in columns:
        Schema += str(column) + "\n"

# Output the Schema and col
print(Schema)

print("\n")

print("OUR TABLE CONTAINS DATA LIKE THIS BELOW:")
print('-' * 80)
# Print data for each dataset
for headers, data in zip(actual_cols, tabdata):
    # Print headers
    print(', '.join(headers))
    
    # Print data rows
    
    for row in data:
        print(', '.join(map(str, row)))
    
    # Print a separator
    print('-' * 80)


In Our DataBase nepa we have Following Tables:
Table: pos

Schema of table `pos`:
Description of Table pos["Field","Type","Null","Key","Default","Extra"] are given below ==> 

('Unnamed: 0', 'bigint(20)', 'YES', '', None, '')
('Date', 'datetime', 'YES', '', None, '')
('Customer', 'text', 'YES', '', None, '')
('CustomerID', 'bigint(20)', 'YES', '', None, '')
('OdooID', 'text', 'YES', '', None, '')
('OrderReference', 'text', 'YES', '', None, '')
('ProductName', 'text', 'YES', '', None, '')
('UnitPrice', 'double', 'YES', '', None, '')
('OrderQuantity', 'bigint(20)', 'YES', '', None, '')
('Subtotal', 'double', 'YES', '', None, '')
('Company', 'text', 'YES', '', None, '')
('ReceiptNumber', 'text', 'YES', '', None, '')
('Total', 'double', 'YES', '', None, '')



OUR TABLE CONTAINS DATA LIKE THIS BELOW:
--------------------------------------------------------------------------------
Unnamed: 0, Date, Customer, CustomerID, OdooID, OrderReference, ProductName, UnitPrice, OrderQuantity, Subtotal

# 2) Code for Generating DataBase Schema in .txt File

In [2]:
import mysql.connector

Schema = ""
col = []

database = 'nepa'
output_file = 'database_info.txt'

# Establish a connection to the database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='nepa'
)

cursor = connection.cursor()

# Fetch the list of tables in the database
dtables = []
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

output_content = [f"In Our DataBase {database} we have Following Tables:\n"]
for i in tables:
    output_content.append(f"Table: {i[0]}\n")
    dtables.append(i[0])

# Initialize the actual_cols to store the list of column names for each table
actual_cols = []
tabdata = []

for table_name in dtables:
    cursor.execute(f"DESCRIBE {table_name}")
    des = cursor.fetchall()

    # Collect column names for the current table
    cols = [column[0] for column in des]
    actual_cols.append(cols)
    
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = cursor.fetchall()
    tabdata.append(rows)

# Loop through each table and describe its schema
for table_name in dtables:
    Schema += f"\nSchema of table `{table_name}`:\n"
    cursor.execute(f"DESCRIBE {table_name}")
    columns = cursor.fetchall()
    
    # Print the header for the schema
    Schema += f'Description of Table {table_name}["Field","Type","Null","Key","Default","Extra"] are given below ==> \n\n'
    
    # Print each column's schema
    for column in columns:
        Schema += str(column) + "\n"

output_content.append(Schema + "\n")
output_content.append("Our table contains data like the following. The data below is not complete but serves as a sample to show what type of data our table stores:\n")
output_content.append('-' * 80 + "\n")
# Print data for each dataset
for headers, data in zip(actual_cols, tabdata):
    # Print headers
    output_content.append(', '.join(headers) + "\n")
    
    # Print data rows
    for row in data:
        output_content.append(', '.join(map(str, row)) + "\n")
    
    # Print a separator
    output_content.append('-' * 80 + "\n")

# Write output to file
with open(output_file, 'w') as file:
    file.writelines(output_content)

print("Database information has been written to", output_file)


Database information has been written to database_info.txt


# 3) SQL Query Code

In [96]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
import re
import mysql.connector
import traceback
from langchain_groq import ChatGroq


def llm_model(key, model, temp=0, typee=None):
    if typee == "groq":
        llm = ChatGroq(model=model, temperature=temp, groq_api_key=key)
    elif typee == "ollama":
        llm = ChatOllama(model=model, temperature=temp, ollama_api_key=key)
    else:
        raise ValueError("Unsupported model type specified.")
    return llm


def make_connection(host, user, password, db):
    # Establish a connection to the database
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=db
    )
    return connection


def generator_prompt(llm):
    # Define prompt chains
    first_prompt = ChatPromptTemplate.from_template(
        template="""
            You are a helpful assistant specialized in generating SQL queries. Your first task is to understand the user's intent based on their question.
            
            User Question: "{user_question}" 
            
            Task: Summarize the user's intent in one sentence."""
    )

    chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="user_intent")

    second_prompt = ChatPromptTemplate.from_template(
        "Given the following database schema: {schema}, generate an SQL query that fulfills the user intent described below:\n\n"
        "User Intent: {user_intent}\n\n"
        "Ensure the query is optimized and accurately reflects the intent."
    )

    chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="prequery")

    third_prompt = ChatPromptTemplate.from_template(
        "From this text, I want only the SQL part text. Remove other unnecessary things: \n\n{prequery}"
    )

    chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="finalsql")

    fourth_prompt = ChatPromptTemplate.from_template(
        "Verify whether the query '{finalsql}' corresponds to the user's question: '{user_question}' based on the given schema: '{schema}'. Provide only the final SQL code."
    )

    chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="ffinalsql")

    # Combine the chains
    overall_chain = SequentialChain(
        chains=[chain_one, chain_two, chain_three, chain_four],
        input_variables=["user_question", "schema"],
        output_variables=["user_intent", "prequery", "finalsql", "ffinalsql"],
        verbose=True
    )

    return overall_chain


def generate_sql(user_question, schema_path, llm):
    # Load the schema from the given path
    with open(schema_path, 'r') as file:
        schema = file.read()

    # Initialize the prompt generator with the schema
    generate_prompt = generator_prompt(llm)

    # Run the SequentialChain with the inputs
    result = generate_prompt.invoke({
        "user_question": user_question,
        "schema": schema
    })

    # Display results for debugging
    print("User Intent:", result.get("user_intent", "Not available"))
    print("-" * 100)
    print("Pre-query:", result.get("prequery", "Not available"))
    print("-" * 100)
    print("Final SQL:", result.get("finalsql", "Not available"))
    print("-" * 100)
    print("Super Final SQL:", result.get("ffinalsql", "Not available"))
    print("-" * 100)

    def extract_clean_sql(text):
        """Helper function to extract and clean SQL query from the provided text."""
        match = re.search(r"SELECT.*?;", text, re.DOTALL)
        if match:
            sql_query = match.group()
            if "\n" in sql_query:
                return ' '.join(sql_query.replace('\n', ' ').split())
            else:
                return sql_query
        return None

    # Try to extract from 'ffinalsql' first
    text = str(result.get("ffinalsql", ""))
    cleaned_query = extract_clean_sql(text)

    if cleaned_query:
        return cleaned_query, schema

    # Fallback to 'finalsql' if not found in 'ffinalsql'
    text = str(result.get("finalsql", ""))
    cleaned_query = extract_clean_sql(text)

    if cleaned_query:
        return cleaned_query, schema

    # If no SQL found in either, print and return None
    print("No SQL query found in the text from both 'ffinalsql' and 'finalsql'.")
    return None, None


def run_sqlgenie(connection, sql_query, schema, llms):
    """
    Attempts to run a SQL query and correct it using a list of LLMs if an error occurs.

    Parameters:
    - connection: The MySQL database connection object.
    - sql_query: The initial SQL query to be executed.
    - schema: The database schema to reference for corrections.
    - llms: A list of LLM instances to use for corrections in sequence.

    Returns:
    - The results of the SQL query if successful, otherwise None.
    """
    try:
        cursor = connection.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        print("SQL Query Results:", result)
        return result
    except Exception as e:
        # Capture the error traceback in a variable
        print("An error occurred while executing the SQL query.")
        error = traceback.format_exc()

        print("\n")
        
        # Iterate through each LLM in the list
        for i, llm in enumerate(llms):
            print(f"Trying corrections with LLM {i + 1}...")

            # Attempt to correct the SQL query up to 3 times with the current LLM
            for attempt in range(3):
                corrected_sql = correct_sql(sql_query, error, schema, llm)
                if corrected_sql:
                    try:
                        cursor.execute(corrected_sql)
                        result = cursor.fetchall()
                        print(f"Corrected SQL Query Results with LLM {i + 1} (Attempt {attempt + 1}):", result)
                        return result
                    except Exception as e:
                        print(f"Corrected query failed on attempt {attempt + 1} with LLM {i + 1}:")

        # If all attempts with all LLMs fail, return None
        print("Failed to correct SQL query after multiple attempts with all LLMs.")
      
        return None

def correct_sql(incorrect_sql, error, schema, llm):
    # Prompt template for correcting SQL
    first_prompt_correct = ChatPromptTemplate.from_template(
        template="""
            You are a SQL queries validator. Your task is to correct incorrect queries. You will always receive an incorrect query as input.

            The incorrect query is: {incorrect_sql}.

            The error is: {err}. 

            The database schema is: {sch}.

            Task: Your job is to fix the incorrect query based on the provided error and refer to the database schema for corrections.
        """
    )

    chain_one_corr = LLMChain(llm=llm, prompt=first_prompt_correct, output_key="cor1")

    second_prompt_correct = ChatPromptTemplate.from_template(
        template="""
            Extract and provide only the SQL code from the following corrected text: {cor1}.
            
            Task: Focus solely on filtering out/ and returning the SQL code, ignoring any other text or information in the paragraphs.
        """
    )

    chain_two_corr = LLMChain(llm=llm, prompt=second_prompt_correct, output_key="ffquery")

    overall_chain_cor = SequentialChain(
        chains=[chain_one_corr, chain_two_corr],
        input_variables=["incorrect_sql", "err", "sch"],
        output_variables=["cor1", "ffquery"],
        verbose=True
    )

    result_cor = overall_chain_cor.invoke({
        "incorrect_sql": incorrect_sql,
        "err": error,
        "sch": schema,
    })

    corrected_text = str(result_cor["ffquery"])

    # Extract corrected SQL query using regex
    match = re.search(r"SELECT.*?;", corrected_text, re.DOTALL)

    if match:
        corrected_sql_query = match.group()
        print("Corrected SQL Query:", corrected_sql_query)
        return corrected_sql_query
    else:
        print("No corrected SQL query found in the text.")
        return None


In [97]:
key="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"

# preparing llms for run

In [98]:

# Define your LLM instances
llm1 = llm_model(key, model="mixtral-8x7b-32768", typee='groq')  # First LLM instance
llm2 = llm_model(key, model="gemma2-9b-it", typee='groq')   # Second LLM instance
llm3 = llm_model(key, model="llama3-8b-8192", typee='groq')  # Third LLM instance

# List of LLMs to use
llms = [llm1, llm2, llm3]

# Connect to your database
conn = make_connection(host='localhost', user="root", password='', db='nepa')


prompt1=generator_prompt(llm1)


In [99]:
user_question=str(input("ENETR YOUR QUERY"))

In [100]:
sql, schema=generate_sql(user_question,"database_info.txt",llm1)



> Entering new SequentialChain chain...

> Finished chain.
User Intent: The user wants to retrieve the top 300 products with the highest number of items sold.
----------------------------------------------------------------------------------------------------
Pre-query: To retrieve the top 300 products with the highest number of items sold, you can use the following SQL query:
```sql
SELECT ProductName, SUM(OrderQuantity) as TotalQuantity
FROM pos
GROUP BY ProductName
ORDER BY TotalQuantity DESC
LIMIT 300;
```
This query groups the data by `ProductName` and calculates the total quantity sold for each product using the `SUM` function. The results are then sorted in descending order based on the total quantity sold and limited to the top 300 products using the `LIMIT` clause.
----------------------------------------------------------------------------------------------------
Final SQL: ```sql
SELECT ProductName, SUM(OrderQuantity) as TotalQuantity
FROM pos
GROUP BY ProductName
ORDER BY

In [101]:
print(sql)

SELECT ProductName, SUM(OrderQuantity) as TotalQuantity FROM pos GROUP BY ProductName ORDER BY TotalQuantity DESC LIMIT 300;


In [102]:
# Run the SQL genie function
results = run_sqlgenie(connection=conn, sql_query=sql, schema=schema, llms=llms)

SQL Query Results: [('hqd plus 5% 1x6pk disposable (1200) (black ice)', Decimal('25916')), ('hqd plus 5% 1x6pk disposable (1200) (sky mint)', Decimal('12011')), ('hqd plus 5% 1x6pk disposable (1200) (ice mint)', Decimal('11071')), ('hqd plus 5% 1x6pk disposable (1200) (grapey)', Decimal('9703')), ('black & mild 25ct (regular)', Decimal('9208')), ('hqd plus 5% 1x6pk disposable (1200) (blueberry lemonade)', Decimal('8643')), ('grabba leaf whole leaf 10 pk', Decimal('8313')), ('fume 10pk extra 5% disposable (1500)  (mint ice)', Decimal('5372')), ('black & mild 10-5pk (regular)', Decimal('5352')), ('y-max big braided i10 1ct', Decimal('5278')), ('hqd plus 5% 1x6pk disposable (1200) (lush ice)', Decimal('5263')), ('grabba leaf small 25 pk', Decimal('4781')), ('fume 5pk infinity plus 5% disposable (4500) (mint ice)', Decimal('4659')), ('hqd plus 5% 1x6pk disposable (1200) (strawberry)', Decimal('4606')), ('hqd plus 5% 1x6pk disposable (1200) (tobacco)', Decimal('4291')), ('opms black 5ct bli

In [103]:
import pandas as pd

In [104]:
results

[('hqd plus 5% 1x6pk disposable (1200) (black ice)', Decimal('25916')),
 ('hqd plus 5% 1x6pk disposable (1200) (sky mint)', Decimal('12011')),
 ('hqd plus 5% 1x6pk disposable (1200) (ice mint)', Decimal('11071')),
 ('hqd plus 5% 1x6pk disposable (1200) (grapey)', Decimal('9703')),
 ('black & mild 25ct (regular)', Decimal('9208')),
 ('hqd plus 5% 1x6pk disposable (1200) (blueberry lemonade)', Decimal('8643')),
 ('grabba leaf whole leaf 10 pk', Decimal('8313')),
 ('fume 10pk extra 5% disposable (1500)  (mint ice)', Decimal('5372')),
 ('black & mild 10-5pk (regular)', Decimal('5352')),
 ('y-max big braided i10 1ct', Decimal('5278')),
 ('hqd plus 5% 1x6pk disposable (1200) (lush ice)', Decimal('5263')),
 ('grabba leaf small 25 pk', Decimal('4781')),
 ('fume 5pk infinity plus 5% disposable (4500) (mint ice)', Decimal('4659')),
 ('hqd plus 5% 1x6pk disposable (1200) (strawberry)', Decimal('4606')),
 ('hqd plus 5% 1x6pk disposable (1200) (tobacco)', Decimal('4291')),
 ('opms black 5ct blister

In [105]:
from tabulate import tabulate

print(tabulate(results))


---------------------------------------------------------------  -----
hqd plus 5% 1x6pk disposable (1200) (black ice)                  25916
hqd plus 5% 1x6pk disposable (1200) (sky mint)                   12011
hqd plus 5% 1x6pk disposable (1200) (ice mint)                   11071
hqd plus 5% 1x6pk disposable (1200) (grapey)                      9703
black & mild 25ct (regular)                                       9208
hqd plus 5% 1x6pk disposable (1200) (blueberry lemonade)          8643
grabba leaf whole leaf 10 pk                                      8313
fume 10pk extra 5% disposable (1500)  (mint ice)                  5372
black & mild 10-5pk (regular)                                     5352
y-max big braided i10 1ct                                         5278
hqd plus 5% 1x6pk disposable (1200) (lush ice)                    5263
grabba leaf small 25 pk                                           4781
fume 5pk infinity plus 5% disposable (4500) (mint ice)            4659
hqd pl

In [108]:
df=pd.DataFrame(columns=['name','quantity'],data=results)

In [109]:
df

,name,quantity
0,hqd plus 5% 1x6pk disposable (1200) (black ice),25916
1,hqd plus 5% 1x6pk disposable (1200) (sky mint),12011
2,hqd plus 5% 1x6pk disposable (1200) (ice mint),11071
3,hqd plus 5% 1x6pk disposable (1200) (grapey),9703
4,black & mild 25ct (regular),9208
...,...,...
295,blunt ville 25 ct $0.99 (triple vanilla),510
296,geek bar pulse 5% 1x5pk disposable (white gumm...,510
297,squeegee with handle,509
298,wrigley's 50c 5-40ct (doublemint),502


In [111]:
df.to_excel("top_300.xlsx",index=False)